This notebooks runs initial tests with hyperparameter tuning and ensures the `tune_nn` function is working as expected.

# Setup

## Basic Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt #for displaying plots
import pandas as pd
import seaborn as sns
import tensorflow as tf

from keras.layers import *
from keras.models import Sequential
from keras.models import Model
from keras.optimizers import *
from keras.callbacks import *
from keras import regularizers
from keras.utils import plot_model

from tensorflow.keras.saving import register_keras_serializable

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.inspection import permutation_importance
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import datetime
import random
import properscoring as ps

from scipy.stats import norm
from scikeras.wrappers import KerasRegressor

import xgboost
import pydot
import graphviz

from keras.models import load_model
from sklearn.multioutput import MultiOutputRegressor
import pickle

from sklearn.model_selection import ParameterGrid
from pathlib import Path
import joblib
import os


In [2]:
# setting a seed to ensure reproducability and consistency
random.seed(16)
np.random.seed(16)
tf.random.set_seed(16)

In [3]:
# Change Working Directory
os.chdir("..")
print(os.getcwd()) # print current working directory

c:\Users\aisti\OneDrive\Dokumente\Uni\Bachelorarbeit\Probabilistic-Forecasting-of-Bicycle-Counts-in-Karlsruhe-with-Neural-Networks


In [4]:
plt.rcParams.update({'font.size': 14})

In [5]:
X_train = pd.read_pickle(r"data\X_train_final.pkl")
X_valid = pd.read_pickle(r"data\X_valid_final.pkl")
X_test = pd.read_pickle(r"data\X_test_final.pkl")

y_train = pd.read_pickle(r"data\y_train_final.pkl")
y_valid = pd.read_pickle(r"data\y_valid_final.pkl")
y_test = pd.read_pickle(r"data\y_test_final.pkl")


In [6]:
X_train.head()

,temperature,precipitation,public_holiday_indicator,is_workday,day_of_week,spring,summer,autumn,winter,windspeed,sun,school_holiday_indicator
0,-0.039776,-0.397192,0,1,2,1.0,0.0,0.0,0.0,-0.218119,-0.031329,0
1,0.244625,-0.397192,0,1,3,1.0,0.0,0.0,0.0,-0.324284,-0.458165,0
2,0.381694,-0.371451,0,1,4,1.0,0.0,0.0,0.0,-1.076810,0.254435,0
3,1.300863,-0.397192,0,0,5,1.0,0.0,0.0,0.0,-0.405469,1.357699,0
4,0.745990,0.555250,0,0,6,1.0,0.0,0.0,0.0,0.297096,0.059103,0


In [7]:
X_valid.tail()

,temperature,precipitation,public_holiday_indicator,is_workday,day_of_week,spring,summer,autumn,winter,windspeed,sun,school_holiday_indicator
359,-1.064496,-0.397192,0,1,1,0.0,0.0,0.0,1.0,-0.258711,-0.353265,1
360,-0.478837,-0.397192,0,1,2,0.0,0.0,0.0,1.0,1.087091,-0.823509,1
361,-0.104279,0.143383,0,1,3,0.0,0.0,0.0,1.0,2.276769,-0.913940,1
362,-0.691404,0.169125,0,1,4,0.0,0.0,0.0,1.0,-0.674006,-0.508807,1
363,0.541485,-0.397192,0,0,5,0.0,0.0,0.0,1.0,1.564836,-0.201340,1


In [8]:
y_test.head()

0    1160.0
1    2996.0
2    3635.0
3    3265.0
4    3751.0
Name: bike_count, dtype: float64

In [9]:
X_train.shape, X_valid.shape, X_test.shape

((3161, 12), (364, 12), (390, 12))

In [10]:
y_train.shape, y_valid.shape, y_test.shape

((3161,), (364,), (390,))

In [11]:
y_train = y_train.reset_index(drop=True)
y_valid = y_valid.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [12]:
y_train.shape, y_valid.shape, y_test.shape

((3161,), (364,), (390,))

In [13]:
X_train.isna().sum()

temperature                 0
precipitation               0
public_holiday_indicator    0
is_workday                  0
day_of_week                 0
spring                      0
summer                      0
autumn                      0
winter                      0
windspeed                   0
sun                         0
school_holiday_indicator    0
dtype: int64

In [14]:
X_train.dtypes

temperature                 float64
precipitation               float64
public_holiday_indicator      int64
is_workday                    int64
day_of_week                   int64
spring                      float64
summer                      float64
autumn                      float64
winter                      float64
windspeed                   float64
sun                         float64
school_holiday_indicator      int64
dtype: object

In [15]:
X_train.describe()

,temperature,precipitation,public_holiday_indicator,is_workday,day_of_week,spring,summer,autumn,winter,windspeed,sun,school_holiday_indicator
count,3.161000e+03,3.161000e+03,3161.000000,3161.000000,3161.000000,3161.000000,3161.000000,3161.000000,3161.000000,3.161000e+03,3.161000e+03,3161.000000
mean,4.450726e-16,3.708939e-17,0.033534,0.684910,3.001898,0.243277,0.259412,0.259095,0.238216,4.832859e-17,4.720467e-17,0.232521
std,1.000158e+00,1.000158e+00,0.180054,0.464625,2.001106,0.429129,0.438381,0.438207,0.426059,1.000158e+00,1.000158e+00,0.422506
min,-2.660948e+00,-3.971923e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.513961e+00,-1.094803e+00,0.000000
25%,-7.932892e-01,-3.971923e-01,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-7.364558e-01,-9.899026e-01,0.000000
50%,-9.270793e-04,-3.971923e-01,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,-2.056287e-01,-2.085744e-01,0.000000
75%,7.907021e-01,-1.140338e-01,0.000000,1.000000,5.000000,0.000000,1.000000,1.000000,0.000000,4.969366e-01,8.585175e-01,0.000000
max,2.566737e+00,1.461021e+01,1.000000,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,4.952762e+00,2.193287e+00,1.000000


## Functions

In [16]:
def add_column(X_train_num, X_valid_num, X_test_num, X_train, X_valid, X_test, col_name):
    
    X_train_num[col_name] = X_train[col_name]
    X_valid_num[col_name] = X_valid[col_name]
    X_test_num[col_name] = X_test[col_name]
    
    return X_train_num, X_valid_num, X_test_num

In [17]:
def crps_normal_cost_function(y_true, y_pred, tensor=True):
    """Compute the CRPS cost function for a normal distribution defined by
    the mean and standard deviation.

    Args:
        y_true: True values
        y_pred: Tensor containing predictions: [mean, std]

    Returns:
        mean_crps: Scalar with mean CRPS over batch or tensor with all crps values of batch
    """
    # get rid of axis 1
    if tensor:
        y_true = y_true[:, 0]
    
    # Split input
    mu = y_pred[:, 0]
    var = tf.square(y_pred[:, 1]) # Ensure standard deviation is positive
    sigma = tf.sqrt(var)  
    #sigma = y_pred[:, 1]

    # The following three variables are just for convenience
    loc = (y_true - mu) / sigma
    phi = 1.0 / tf.sqrt(2.0 * np.pi) * tf.exp(-tf.square(loc) / 2.0)
    Phi = 0.5 * (1.0 + tf.math.erf(loc / tf.sqrt(2.0)))
    # First we will compute the crps for each input/target pair
    crps =  sigma * (loc * (2. * Phi - 1.) + 2 * phi - 1. / tf.sqrt(np.pi))
    # Then we take the mean. The cost is now a scalar
    return tf.reduce_mean(crps) if tensor else crps

In [18]:
def crps_normal_cost_function_v2(y_true, y_pred, tensor=True):
    """Compute the CRPS cost function for a normal distribution defined by
    the mean and standard deviation.

    Args:
        y_true: True values
        y_pred: Tensor containing predictions: [mean, std]

    Returns:
        mean_crps: Scalar with mean CRPS over batch or tensor with all crps values of batch
    """
    # get rid of axis 1
    if tensor:
        y_true = y_true[:, 0]

    # Split input
    mu = y_pred[:, 0]
    sigma = tf.math.softplus(y_pred[:, 1])  # Apply softplus to ensure positive std
    #sigma = y_pred[:, 1]

    # The following three variables are just for convenience
    loc = (y_true - mu) / sigma
    phi = 1.0 / tf.sqrt(2.0 * np.pi) * tf.exp(-tf.square(loc) / 2.0)
    Phi = 0.5 * (1.0 + tf.math.erf(loc / tf.sqrt(2.0)))
    # First we will compute the crps for each input/target pair
    crps =  sigma * (loc * (2. * Phi - 1.) + 2 * phi - 1. / tf.sqrt(np.pi))
    
    # Then we take the mean. The cost is now a scalar
    return tf.reduce_mean(crps) if tensor else crps

In [19]:
def predict_and_calculate_mean_crps(model, X_test = X_test, y_test = y_test, print_predicted_mean_std = False):
    """Make predictions on test data and calculate mean CRPS.

    Args:
        model: Trained model
        X_test: Test data
        y_test: True values
        print_predicted_mean_std: Boolean to print predicted mean and std

    Returns:
        mean_crps: Scalar with mean CRPS over batch or tensor with all crps values of batch
    """
     
    predicted_mean_std = model.predict(X_test)
    if print_predicted_mean_std:
        print(predicted_mean_std)
    
    predicted_mean = predicted_mean_std[:, 0]
    predicted_std = predicted_mean_std[:, 1]
    mean_crps = np.mean(ps.crps_gaussian(y_test, mu=predicted_mean, sig=predicted_std))
    print(f"\nMean CRPS: { mean_crps }")


    return mean_crps, predicted_mean_std

In [20]:
def compile_and_fit_model(model, X_train = X_train, y_train = y_train, X_valid = X_valid, y_valid = y_valid,  pat=5, lr=1e-3, loss_function = crps_normal_cost_function, epochs = 100, batch_size = 10):
    """Compile and fit a model on training data.

    Args:
        model: model to be trained
        X_train: Training data
        y_train: Training targets
        X_valid: Validation data
        y_valid: Validation targets
        pat: Patience for early stopping
        lr: Learning rate
        loss_function: Loss function
        epochs: Number of epochs
        batch_size: Batch size

    Returns:
        model: Trained model
    """

    #print some information about the model
    print('Model information:\n\nCallbacks: EarlyStopping (patience: ' + str(pat) + '), Learning Rate (Reduce on Plateau, patience: ' + str(pat) + ')\nBatch size: ' + str(batch_size) + '\nEpochs: ' + str(epochs) + '\n')
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=pat, 
                                                      restore_best_weights=True)
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                     factor=0.2,
                                                     patience = pat,
                                                     min_lr=0 )
    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss=loss_function)

    # Train the model
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid), callbacks=[early_stopping, reduce_lr])

    return model, history

In [21]:
def plot_loss(history):
    """Plot training and validation loss.

    Args:
        history: History object
    """
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training loss')
    plt.plot(history.history['val_loss'], label='Validation loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()

In [22]:
def process_model(model, X_train = X_train, y_train = y_train, X_valid = X_valid, y_valid = y_valid,  X_test = X_test, y_test = y_test, pat=5, lr=1e-3, loss_function = crps_normal_cost_function, epochs = 100, batch_size = 10, print_predicted_mean_std = False, plot_loss_flag = True):
    """Compile, fit and evaluate a model.

    Args:
        model: model to be trained
        X_train: Training data
        y_train: Training targets
        X_valid: Validation data
        y_valid: Validation targets
        X_test: Test data
        y_test: Test targets
        pat: Patience for early stopping
        lr: Learning rate
        loss_function: Loss function
        epochs: Number of epochs
        batch_size: Batch size
        print_predicted_mean_std: Boolean to print predicted mean and std
"""

    model, history = compile_and_fit_model(model, X_train, y_train, X_valid, y_valid,  pat, lr, loss_function, epochs, batch_size)

    mean_crps, predicted_mean_std = predict_and_calculate_mean_crps(model, X_test, y_test, print_predicted_mean_std)

    if plot_loss_flag:
        plot_loss(history)

    return model, mean_crps, history, predicted_mean_std

# Hyperparameter Tuning

In [23]:
@register_keras_serializable(package="Custom", name="crps_normal_cost_function")
def crps_normal_cost_function_v3(y_true, y_pred, tensor=True):
    """Compute the CRPS cost function for a normal distribution defined by
    the mean and standard deviation.

    Args:
        y_true: True values
        y_pred: Tensor containing predictions: [mean, std]

    Returns:
        mean_crps: Scalar with mean CRPS over batch or tensor with all crps values of batch
    """
    #print("y_true shape2: ", y_true.shape)
    # Split input
    mu = y_pred[:, 0]
    sigma = tf.math.softplus(y_pred[:, 1])  # Apply softplus to ensure positive std
    #sigma = y_pred[:, 1]
    
    # The following three variables are just for convenience
    loc = (y_true - mu) / sigma
    phi = 1.0 / tf.sqrt(2.0 * np.pi) * tf.exp(-tf.square(loc) / 2.0)
    Phi = 0.5 * (1.0 + tf.math.erf(loc / tf.sqrt(2.0)))
    
     # First we will compute the crps for each input/target pair
    crps =  sigma * (loc * (2. * Phi - 1.) + 2 * phi - 1. / tf.sqrt(np.pi))   
    
    # Then we take the mean. The cost is now a scalar
    return tf.reduce_mean(crps) if tensor else crps


In [24]:
@register_keras_serializable(package="Custom", name="crps_loss_function_wrapper")  
def crps_loss_function_wrapper(y_true, y_pred, tensor=True):
    """Wrapper for CRPS loss function.

    Args:
        y_true: True values
        y_pred: Tensor containing predictions: [mean, std]

    Returns:
        mean_crps: Scalar with mean CRPS over batch
    """
    
    if isinstance(y_pred, (pd.DataFrame, pd.Series)):
       y_pred = tf.convert_to_tensor(y_pred.values)

    if isinstance(y_true, (pd.DataFrame, pd.Series)):
        y_true = tf.convert_to_tensor(y_true.values, dtype=tf.float32)
        
    y_pred_reshaped = y_pred.reshape(-1, 2) #Keras Regressor flattens the values so they need to be reshaped

    crps = crps_normal_cost_function_v3(y_true, y_pred_reshaped, tensor)

    crps = crps.numpy()
    
    return crps

In [25]:
def tune_nn(input_shape_in, X_train_in, X_valid_in, X_test_in, y_train_in, y_valid_in, y_test_in, model_in=None, n_neurons=[64, 128, 256], dropout_rate=[0.1, 0.3, 0.5], 
            reg_values=[0.01], batch_size=[32], learning_rate=[1e-3, 1e-4], n_blocks=[1,2,4], epochs = 200, cv=5, n_jobs=1, verbose=10, patience=5):
    """Tune hyperparameters of a neural network using GridSearchCV.

    Args:
        input_shape_in: Number of features
        X_train_in: Training data
        X_valid_in: Validation data
        X_test_in: Test data
        y_train_in: Training targets
        y_valid_in: Validation targets
        y_test_in: Test targets
        model_in: Predefined model
        n_neurons: Number of neurons in hidden layers
        dropout_rate: Dropout rate
        reg_values: Regularization values
        batch_size: Batch size
        learning_rate: Learning rate
        n_blocks: Number of blocks
        epochs: Number of epochs
        cv: Number of cross-validation folds


    Returns:
        y_pred: Predictions on test data
        best_model: Best model
        cv_results: Results of cross-validation
    """


    iteration = 0

    # Define Keras model with hyperparameters
    def create_model(n_neurons, dropout_rate, reg_value, n_blocks):
        nonlocal iteration  # Declare iteration as nonlocal
        model = model_in
        input_shape = input_shape_in
        if model is None:
            model = Sequential()
            model.add(Input(shape=(input_shape,)))

            for i in range(n_blocks):
                model.add(Dense(n_neurons, activation='relu', kernel_regularizer=regularizers.l2(reg_value)))
                model.add(Dense(n_neurons, activation='relu'))
                model.add(Dropout(dropout_rate))
            
            # Output layer with two outputs: mean and std
            model.add(Dense(n_neurons, activation='relu', kernel_regularizer=regularizers.l2(reg_value)))
            model.add(Dense(2, activation='softplus'))
        
        if iteration % cv == 1 or iteration == 0: # don't print for every fold
            model.summary()
        
        iteration += 1
        # only to see that something is happening
        print("Iteration: " + str(iteration))

        
        model.compile(optimizer=tf.keras.optimizers.Adam(), loss=crps_normal_cost_function_v3 )
        return model

    # Define hyperparameters
    param_grid = dict(
       batch_size=batch_size,
       optimizer__learning_rate=learning_rate,
        model__n_neurons=n_neurons,
        model__dropout_rate=dropout_rate,
        model__reg_value=reg_values,
        model__n_blocks=n_blocks
    ) 


    # Create the model
    model = KerasRegressor(model=create_model, epochs=epochs, verbose=0)
    

    my_scorer = make_scorer(crps_loss_function_wrapper, greater_is_better=False) 

    # Perform GridSearchCV
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, n_jobs=n_jobs, verbose=verbose, scoring=my_scorer)

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    
    grid_search.fit(X_train_in, y_train_in, validation_data=(X_valid_in, y_valid_in), callbacks=[early_stopping])

    # Access the best hyperparameters and results
    best_params = grid_search.best_params_
    cv_results = pd.DataFrame(grid_search.cv_results_)
    best_model = grid_search.best_estimator_

    # Print the best hyperparameters
    print("Best Hyperparameters:")
    for param, value in best_params.items():
        print(f"{param}: {value}")

    y_pred = best_model.model_.predict(X_test_in)
    predicted_mean = y_pred[:, 0]
    predicted_std = y_pred[:, 1]

    mean_crps = np.mean(ps.crps_gaussian(y_test_in, mu=predicted_mean, sig=predicted_std))
    print(f"\nMean CRPS: { mean_crps }")
    
    return  y_pred, best_model, cv_results, grid_search
    

In [26]:
y_pred_tuned_test, best_model_tuned_test, cv_results_tuned_test, grid_search_tuned_test  = tune_nn(X_train.shape[1], X_train, X_valid, X_test, y_train, y_valid, y_test, n_neurons=[256], dropout_rate=[0.2], reg_values=[0.01], batch_size=[64], learning_rate=[1e-4], n_blocks=[4], epochs=2)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START batch_size=64, model__dropout_rate=0.2, model__n_blocks=4, model__n_neurons=256, model__reg_value=0.01, optimizer__learning_rate=0.0001


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │         3,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 530,178 (2.02 MB)

 Trainable params: 530,178 (2.02 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 256)            │         3,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 530,178 (2.02 MB)

 Trainable params: 530,178 (2.02 MB)

 Non-trainable params: 0 (0.00 B)

Iteration: 2
[CV 2/5; 1/1] END batch_size=64, model__dropout_rate=0.2, model__n_blocks=4, model__n_neurons=256, model__reg_value=0.01, optimizer__learning_rate=0.0001;, score=-696.268 total time=   7.0s
[CV 3/5; 1/1] START batch_size=64, model__dropout_rate=0.2, model__n_blocks=4, model__n_neurons=256, model__reg_value=0.01, optimizer__learning_rate=0.0001
Iteration: 3
[CV 3/5; 1/1] END batch_size=64, model__dropout_rate=0.2, model__n_blocks=4, model__n_neurons=256, model__reg_value=0.01, optimizer__learning_rate=0.0001;, score=-609.408 total time=   7.0s
[CV 4/5; 1/1] START batch_size=64, model__dropout_rate=0.2, model__n_blocks=4, model__n_neurons=256, model__reg_value=0.01, optimizer__learning_rate=0.0001
Iteration: 4
[CV 4/5; 1/1] END batch_size=64, model__dropout_rate=0.2, model__n_blocks=4, model__n_neurons=256, model__reg_value=0.01, optimizer__learning_rate=0.0001;, score=-655.296 total time=   6.6s
[CV 5/5; 1/1] START batch_size=64, model__dropout_rate=0.2, model__n_blocks=4, 

In [27]:
joblib.dump(grid_search_tuned_test, r"hyperparameter_tuning_results\grid_search_tuned_test.pkl")

['hyperparameter_tuning_results\\grid_search_tuned_test.pkl']

In [28]:
loaded_grid_search_tuned_test = joblib.load(r"hyperparameter_tuning_results\grid_search_tuned_test.pkl")

In [29]:
loaded_grid_search_tuned_test.best_params_

{'batch_size': 64,
 'model__dropout_rate': 0.2,
 'model__n_blocks': 4,
 'model__n_neurons': 256,
 'model__reg_value': 0.01,
 'optimizer__learning_rate': 0.0001}

In [30]:
cv_results_tuned_test

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_model__dropout_rate,param_model__n_blocks,param_model__n_neurons,param_model__reg_value,param_optimizer__learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,6.356501,0.421882,0.363018,0.04554,64,0.2,4,256,0.01,0.0001,"{'batch_size': 64, 'model__dropout_rate': 0.2,...",-640.987244,-696.2677,-609.408447,-655.295959,-632.644226,-646.920715,28.814802,1


In [31]:
y_pred_tuned_test

array([[1141.743   ,  243.39665 ],
       [3559.3667  ,  758.7549  ],
       [3942.6667  ,  840.44073 ],
       [3591.9214  ,  765.67285 ],
       [3692.931   ,  787.2072  ],
       [ 662.8439  ,  141.34319 ],
       [1494.0311  ,  318.47614 ],
       [1253.7742  ,  267.27457 ],
       [3168.7053  ,  675.48987 ],
       [4126.387   ,  879.6275  ],
       [3195.0273  ,  681.0932  ],
       [4473.682   ,  953.6151  ],
       [3377.3628  ,  719.92694 ],
       [1552.0258  ,  330.85458 ],
       [1116.4214  ,  238.0007  ],
       [3144.0574  ,  670.2343  ],
       [4164.5273  ,  887.7466  ],
       [3793.3884  ,  808.6283  ],
       [3790.864   ,  808.0861  ],
       [3964.3777  ,  845.059   ],
       [1246.1832  ,  265.6641  ],
       [1491.6031  ,  317.96448 ],
       [3554.4033  ,  757.6978  ],
       [3743.201   ,  797.94415 ],
       [3860.1929  ,  822.87195 ],
       [4030.4727  ,  859.1547  ],
       [3980.1714  ,  848.4272  ],
       [1355.2256  ,  288.90668 ],
       [1612.2594  ,

In [32]:
y_pred_tuned_test.shape

(390, 2)

In [33]:
y_pred_tuned, best_model_tuned, cv_results_tuned, grid_search_tuned = tune_nn(X_train.shape[1], X_train, X_valid, X_test, y_train, y_valid, y_test, cv=3 )

Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV 1/3; 1/54] START batch_size=32, model__dropout_rate=0.1, model__n_blocks=1, model__n_neurons=64, model__reg_value=0.01, optimizer__learning_rate=0.001


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_60 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,282 (36.26 KB)

 Trainable params: 9,282 (36.26 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1120 (Dense)              │ (None, 256)            │         3,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1121 (Dense)              │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_394 (Dropout)           │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1122 (Dense)              │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1123 (Dense)              │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_395 (Dropout)           │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1124 (Dense)              │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1125 (Dense)              │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_396 (Dropout)           │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1126 (Dense)              │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1127 (Dense)              │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_397 (Dropout)           │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1128 (Dense)              │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1129 (Dense)              │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 530,178 (2.02 MB)

 Trainable params: 530,178 (2.02 MB)

 Non-trainable params: 0 (0.00 B)

Iteration: 161
[CV 2/3; 54/54] END batch_size=32, model__dropout_rate=0.5, model__n_blocks=4, model__n_neurons=256, model__reg_value=0.01, optimizer__learning_rate=0.0001;, score=-3218.990 total time= 1.6min
[CV 3/3; 54/54] START batch_size=32, model__dropout_rate=0.5, model__n_blocks=4, model__n_neurons=256, model__reg_value=0.01, optimizer__learning_rate=0.0001
Iteration: 162
[CV 3/3; 54/54] END batch_size=32, model__dropout_rate=0.5, model__n_blocks=4, model__n_neurons=256, model__reg_value=0.01, optimizer__learning_rate=0.0001;, score=-2180.141 total time= 1.6min
Iteration: 163
Best Hyperparameters:
batch_size: 32
model__dropout_rate: 0.3
model__n_blocks: 1
model__n_neurons: 128
model__reg_value: 0.01
optimizer__learning_rate: 0.0001
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

Mean CRPS: 324.37335963573844


In [34]:
y_pred_tuned

array([[ 570.6501 ,  135.0581 ],
       [2814.6338 ,  506.32217],
       [3090.5508 ,  506.15286],
       [2928.7498 ,  460.1286 ],
       [3225.8735 ,  471.10034],
       [ 789.02826,  140.46143],
       [2239.047  ,  350.83524],
       [1185.0128 ,  249.67908],
       [3678.989  ,  611.90295],
       [4668.653  ,  669.68317],
       [3824.5735 ,  560.9098 ],
       [4620.4985 ,  641.3632 ],
       [3792.0657 ,  542.7761 ],
       [3322.592  ,  484.03244],
       [1168.9683 ,  252.35327],
       [3621.7576 ,  605.8409 ],
       [4611.4004 ,  663.0519 ],
       [4074.5928 ,  583.9914 ],
       [4121.8687 ,  578.7114 ],
       [4282.455  ,  600.7206 ],
       [2517.1511 ,  393.69144],
       [1262.2915 ,  263.6888 ],
       [3988.3645 ,  634.49567],
       [4145.4814 ,  616.7096 ],
       [4201.1694 ,  597.3205 ],
       [4167.6353 ,  584.51294],
       [4170.156  ,  587.14734],
       [2879.2256 ,  427.69843],
       [1377.3815 ,  282.2382 ],
       [3784.952  ,  613.155  ],
       [41

In [35]:
# Define the base path using pathlib
base_path = Path(r"hyperparameter_tuning_results")

In [36]:
file_name_predictions = "y_pred_tuned.csv"
file_path_predictions = base_path / file_name_predictions

np.savetxt(str(file_path_predictions), y_pred_tuned, delimiter=',')

In [37]:
file_name_cv_results = "cv_results_tuned.csv"
file_path_cv_results = base_path / file_name_cv_results
cv_results_tuned.to_csv(str(file_path_cv_results), index=False)


In [38]:
model_path = base_path / "best_model_tuned.keras"
best_model_tuned.model_.save(str(model_path))

In [39]:
file_name_grid_search = "grid_search_tuned.pkl"
file_path_grid_search = base_path / file_name_grid_search

joblib.dump(grid_search_tuned, str(file_path_grid_search))

['hyperparameter_tuning_results\\grid_search_tuned.pkl']

In [40]:
loaded_grid_search_tuned = joblib.load(str(file_path_grid_search))

In [41]:
loaded_grid_search_tuned.best_params_

{'batch_size': 32,
 'model__dropout_rate': 0.3,
 'model__n_blocks': 1,
 'model__n_neurons': 128,
 'model__reg_value': 0.01,
 'optimizer__learning_rate': 0.0001}

In [42]:
pd.DataFrame(loaded_grid_search_tuned.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_model__dropout_rate,param_model__n_blocks,param_model__n_neurons,param_model__reg_value,param_optimizer__learning_rate,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,59.275715,0.521988,0.258031,0.021568,32,0.1,1,64,0.01,0.0010,"{'batch_size': 32, 'model__dropout_rate': 0.1,...",-418.974670,-358.730438,-447.029572,-408.244893,36.837755,10
1,63.829903,5.137753,0.304035,0.036598,32,0.1,1,64,0.01,0.0001,"{'batch_size': 32, 'model__dropout_rate': 0.1,...",-410.018921,-455.061920,-437.332428,-434.137756,18.526960,16
2,67.961705,1.417156,0.280116,0.031455,32,0.1,1,128,0.01,0.0010,"{'batch_size': 32, 'model__dropout_rate': 0.1,...",-418.317566,-350.799103,-435.880859,-401.665843,36.675934,6
3,64.883362,2.839101,0.312872,0.054113,32,0.1,1,128,0.01,0.0001,"{'batch_size': 32, 'model__dropout_rate': 0.1,...",-405.809357,-352.826660,-437.509735,-398.715251,34.933752,4
4,80.787429,2.247877,0.284046,0.016451,32,0.1,1,256,0.01,0.0010,"{'batch_size': 32, 'model__dropout_rate': 0.1,...",-408.439270,-343.090637,-482.102875,-411.210927,56.785339,12
5,67.223817,7.370789,0.237983,0.005538,32,0.1,1,256,0.01,0.0001,"{'batch_size': 32, 'model__dropout_rate': 0.1,...",-414.825348,-353.893951,-455.412903,-408.044067,41.721407,9
6,69.646406,7.249292,0.320396,0.010844,32,0.1,2,64,0.01,0.0010,"{'batch_size': 32, 'model__dropout_rate': 0.1,...",-409.503723,-375.679352,-520.531738,-435.238271,61.872200,17
7,63.746693,5.095853,0.404434,0.090634,32,0.1,2,64,0.01,0.0001,"{'batch_size': 32, 'model__dropout_rate': 0.1,...",-425.379211,-359.454163,-482.026154,-422.286509,50.087569,14
8,79.683305,3.627351,0.360418,0.035987,32,0.1,2,128,0.01,0.0010,"{'batch_size': 32, 'model__dropout_rate': 0.1,...",-430.512573,-382.856262,-495.918945,-436.429260,46.346866,19
9,88.073267,0.450052,0.374046,0.025498,32,0.1,2,128,0.01,0.0001,"{'batch_size': 32, 'model__dropout_rate': 0.1,...",-443.348083,-359.537872,-543.670349,-448.852102,75.272451,22


In [43]:
# Load the model with custom objects
loaded_model = load_model(
    model_path,
    custom_objects={'crps_normal_cost_function_v3': crps_normal_cost_function_v3}
)

In [44]:
loaded_model.summary()

Model: "sequential_168"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1140 (Dense)              │ (None, 128)            │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1141 (Dense)              │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_402 (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1142 (Dense)              │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1143 (Dense)              │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 104,840 (409.54 KB)

 Trainable params: 34,946 (136.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 69,894 (273.03 KB)

In [45]:
mean_crps, predicted_mean_std = predict_and_calculate_mean_crps(loaded_model, X_test, y_test, print_predicted_mean_std=False)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

Mean CRPS: 324.37335963573844


In [46]:
predicted_mean_std

array([[ 570.6501 ,  135.0581 ],
       [2814.6338 ,  506.32217],
       [3090.5508 ,  506.15286],
       [2928.7498 ,  460.1286 ],
       [3225.8735 ,  471.10034],
       [ 789.02826,  140.46143],
       [2239.047  ,  350.83524],
       [1185.0128 ,  249.67908],
       [3678.989  ,  611.90295],
       [4668.653  ,  669.68317],
       [3824.5735 ,  560.9098 ],
       [4620.4985 ,  641.3632 ],
       [3792.0657 ,  542.7761 ],
       [3322.592  ,  484.03244],
       [1168.9683 ,  252.35327],
       [3621.7576 ,  605.8409 ],
       [4611.4004 ,  663.0519 ],
       [4074.5928 ,  583.9914 ],
       [4121.8687 ,  578.7114 ],
       [4282.455  ,  600.7206 ],
       [2517.1511 ,  393.69144],
       [1262.2915 ,  263.6888 ],
       [3988.3645 ,  634.49567],
       [4145.4814 ,  616.7096 ],
       [4201.1694 ,  597.3205 ],
       [4167.6353 ,  584.51294],
       [4170.156  ,  587.14734],
       [2879.2256 ,  427.69843],
       [1377.3815 ,  282.2382 ],
       [3784.952  ,  613.155  ],
       [41